# DE300 Homework 2

# Please run all cells in order to view the results of the heart disease prediction model.

Importing necessary libraries

In [18]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
import requests
from bs4 import BeautifulSoup
import json
#!pip install scrapy
#!pip install scikit-learn
#import scrapy
from scrapy import Selector

#from scrapy import Selector
#import requests

Reading in csv file as a pandas dataframe

In [19]:
df = pd.read_csv('../rawdata/heart_disease.csv')
df = df.drop(df.index[-2:])

Selecting desired columns from dataframe

In [20]:
# Selecting only the desired columns
selected_columns = ['age', 'sex', 'painloc', 'painexer', 'cp', 'trestbps', 'smoke', 'fbs', 'prop', 'nitr', 'pro', 'diuretic', 'thaldur', 'thalach', 'exang', 'oldpeak', 'slope', 'target']

# Using loc method to select only the desired columns
df = df.loc[:, selected_columns]

df

,age,sex,painloc,painexer,cp,trestbps,smoke,fbs,prop,nitr,pro,diuretic,thaldur,thalach,exang,oldpeak,slope,target
0,63.0,1.0,NaN,NaN,1.0,145.0,NaN,1.0,0.0,0.0,0.0,0.0,10.5,150.0,0.0,2.3,3.0,0.0
1,67.0,1.0,NaN,NaN,4.0,160.0,NaN,0.0,1.0,0.0,0.0,0.0,9.5,108.0,1.0,1.5,2.0,1.0
2,67.0,1.0,NaN,NaN,4.0,120.0,NaN,0.0,1.0,0.0,0.0,0.0,8.5,129.0,1.0,2.6,2.0,1.0
3,37.0,1.0,NaN,NaN,3.0,130.0,NaN,0.0,1.0,0.0,0.0,0.0,13.0,187.0,0.0,3.5,3.0,0.0
4,41.0,0.0,NaN,NaN,2.0,130.0,NaN,0.0,0.0,0.0,0.0,0.0,7.0,172.0,0.0,1.4,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,54.0,1.0,1.0,1.0,4.0,180.0,NaN,NaN,0.0,0.0,0.0,0.0,7.5,150.0,0.0,1.5,2.0,1.0
895,56.0,1.0,1.0,1.0,4.0,125.0,1.0,1.0,0.0,0.0,0.0,0.0,6.5,103.0,1.0,1.0,2.0,1.0
896,56.0,1.0,0.0,1.0,3.0,125.0,NaN,NaN,0.0,0.0,0.0,0.0,6.0,98.0,0.0,-2.0,2.0,1.0
897,54.0,1.0,1.0,1.0,4.0,130.0,NaN,NaN,0.0,0.0,0.0,0.0,9.5,110.0,1.0,3.0,2.0,1.0


Code to check if columns if 'filled' or not

In [21]:
filled_cells = df['target'].count()
filled_cells

899

Code to clean and impute binary attributes

In [22]:
binary_attributes = [
    'painloc',      # Chest pain location (1 = substernal, 0 = otherwise)
    'painexer',     # Whether pain is provoked by exertion (1 = yes, 0 = no)
    'fbs',          # Fasting blood sugar > 120 mg/dL (1 = true, 0 = false)
    'prop',         # Beta blocker used during exercise ECG (1 = yes, 0 = no)
    'nitr',         # Nitrates used during exercise ECG (1 = yes, 0 = no)
    'pro',          # Calcium channel blocker used during exercise ECG (1 = yes, 0 = no)
    'diuretic',     # Diuretic used during exercise ECG (1 = yes, 0 = no)
    'exang'
]

for column in binary_attributes:
    # if np.all(np.isin(df[column].dropna().unique(), [0, 1])):  # Check if only contains 0 or 1
    #     # Step 2: Calculate the mode of the binary column
    #     mode_value = df[column].mode()[0]

    #     # Step 3: Replace outliers
    #     # Outliers are values that are not 0 or 1
    #     df[column] = df[column].apply(lambda x: mode_value if x not in [0, 1] else x)

    # NEW
    # Step 1: Calculate the mode of the binary column. Mode returns a Series, so use [0] to get the value.
    mode_value = df[column].mode()[0]

    # Step 2: Replace non-binary and NaN values with the mode
    # Use apply() to replace values
    df[column] = df[column].apply(lambda x: mode_value if x not in [0, 1] or pd.isna(x) else x)

df

,age,sex,painloc,painexer,cp,trestbps,smoke,fbs,prop,nitr,pro,diuretic,thaldur,thalach,exang,oldpeak,slope,target
0,63.0,1.0,1.0,1.0,1.0,145.0,NaN,1.0,0.0,0.0,0.0,0.0,10.5,150.0,0.0,2.3,3.0,0.0
1,67.0,1.0,1.0,1.0,4.0,160.0,NaN,0.0,1.0,0.0,0.0,0.0,9.5,108.0,1.0,1.5,2.0,1.0
2,67.0,1.0,1.0,1.0,4.0,120.0,NaN,0.0,1.0,0.0,0.0,0.0,8.5,129.0,1.0,2.6,2.0,1.0
3,37.0,1.0,1.0,1.0,3.0,130.0,NaN,0.0,1.0,0.0,0.0,0.0,13.0,187.0,0.0,3.5,3.0,0.0
4,41.0,0.0,1.0,1.0,2.0,130.0,NaN,0.0,0.0,0.0,0.0,0.0,7.0,172.0,0.0,1.4,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,54.0,1.0,1.0,1.0,4.0,180.0,NaN,0.0,0.0,0.0,0.0,0.0,7.5,150.0,0.0,1.5,2.0,1.0
895,56.0,1.0,1.0,1.0,4.0,125.0,1.0,1.0,0.0,0.0,0.0,0.0,6.5,103.0,1.0,1.0,2.0,1.0
896,56.0,1.0,0.0,1.0,3.0,125.0,NaN,0.0,0.0,0.0,0.0,0.0,6.0,98.0,0.0,-2.0,2.0,1.0
897,54.0,1.0,1.0,1.0,4.0,130.0,NaN,0.0,0.0,0.0,0.0,0.0,9.5,110.0,1.0,3.0,2.0,1.0


Code to clean and impute continuous attributes

In [23]:
# Replace missing values in 'thaldur' column with the average of the column
thaldur_average = df['thaldur'].mean()
df['thaldur'].fillna(thaldur_average, inplace=True)

# Replace missing values in 'thalach' column with the average of the column
thalach_average = df['thalach'].mean()
df['thalach'].fillna(thalach_average, inplace=True)

# Replace missing values in 'trestbps' column with the average of the column
thalach_average = df['trestbps'].mean()
df['trestbps'].fillna(thalach_average, inplace=True)

# Calculate the average of the 'oldpeak' column
average_oldpeak = df['oldpeak'].mean()

# Replace missing values, values less than 0, and values greater than 4 with the average
df['oldpeak'] = df['oldpeak'].apply(lambda x: average_oldpeak if pd.isnull(x) or x < 0 or x > 4 else x)

df

/tmp/ipykernel_29/1394458546.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['thaldur'].fillna(thaldur_average, inplace=True)
/tmp/ipykernel_29/1394458546.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

,age,sex,painloc,painexer,cp,trestbps,smoke,fbs,prop,nitr,pro,diuretic,thaldur,thalach,exang,oldpeak,slope,target
0,63.0,1.0,1.0,1.0,1.0,145.0,NaN,1.0,0.0,0.0,0.0,0.0,10.5,150.0,0.0,2.30000,3.0,0.0
1,67.0,1.0,1.0,1.0,4.0,160.0,NaN,0.0,1.0,0.0,0.0,0.0,9.5,108.0,1.0,1.50000,2.0,1.0
2,67.0,1.0,1.0,1.0,4.0,120.0,NaN,0.0,1.0,0.0,0.0,0.0,8.5,129.0,1.0,2.60000,2.0,1.0
3,37.0,1.0,1.0,1.0,3.0,130.0,NaN,0.0,1.0,0.0,0.0,0.0,13.0,187.0,0.0,3.50000,3.0,0.0
4,41.0,0.0,1.0,1.0,2.0,130.0,NaN,0.0,0.0,0.0,0.0,0.0,7.0,172.0,0.0,1.40000,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,54.0,1.0,1.0,1.0,4.0,180.0,NaN,0.0,0.0,0.0,0.0,0.0,7.5,150.0,0.0,1.50000,2.0,1.0
895,56.0,1.0,1.0,1.0,4.0,125.0,1.0,1.0,0.0,0.0,0.0,0.0,6.5,103.0,1.0,1.00000,2.0,1.0
896,56.0,1.0,0.0,1.0,3.0,125.0,NaN,0.0,0.0,0.0,0.0,0.0,6.0,98.0,0.0,0.87049,2.0,1.0
897,54.0,1.0,1.0,1.0,4.0,130.0,NaN,0.0,0.0,0.0,0.0,0.0,9.5,110.0,1.0,3.00000,2.0,1.0


Code to clean and impute categorical attributes

In [24]:
valid_categories = {
    'cp': {1, 2, 3, 4},
    'slope': {1,2,3},
}

for column, valid_set in valid_categories.items():
    mode_value = df[column].mode()[0]
    df[column] = df[column].apply(lambda x: mode_value if x not in valid_set else x)

df

,age,sex,painloc,painexer,cp,trestbps,smoke,fbs,prop,nitr,pro,diuretic,thaldur,thalach,exang,oldpeak,slope,target
0,63.0,1.0,1.0,1.0,1.0,145.0,NaN,1.0,0.0,0.0,0.0,0.0,10.5,150.0,0.0,2.30000,3.0,0.0
1,67.0,1.0,1.0,1.0,4.0,160.0,NaN,0.0,1.0,0.0,0.0,0.0,9.5,108.0,1.0,1.50000,2.0,1.0
2,67.0,1.0,1.0,1.0,4.0,120.0,NaN,0.0,1.0,0.0,0.0,0.0,8.5,129.0,1.0,2.60000,2.0,1.0
3,37.0,1.0,1.0,1.0,3.0,130.0,NaN,0.0,1.0,0.0,0.0,0.0,13.0,187.0,0.0,3.50000,3.0,0.0
4,41.0,0.0,1.0,1.0,2.0,130.0,NaN,0.0,0.0,0.0,0.0,0.0,7.0,172.0,0.0,1.40000,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,54.0,1.0,1.0,1.0,4.0,180.0,NaN,0.0,0.0,0.0,0.0,0.0,7.5,150.0,0.0,1.50000,2.0,1.0
895,56.0,1.0,1.0,1.0,4.0,125.0,1.0,1.0,0.0,0.0,0.0,0.0,6.5,103.0,1.0,1.00000,2.0,1.0
896,56.0,1.0,0.0,1.0,3.0,125.0,NaN,0.0,0.0,0.0,0.0,0.0,6.0,98.0,0.0,0.87049,2.0,1.0
897,54.0,1.0,1.0,1.0,4.0,130.0,NaN,0.0,0.0,0.0,0.0,0.0,9.5,110.0,1.0,3.00000,2.0,1.0


Code to Webscrape ABS URL

In [25]:
# Set the webpage URL for fetching data
data_url = "https://www.abs.gov.au/statistics/health/health-conditions-and-risks/smoking-and-vaping/latest-release"
# Send a GET request to the URL
web_response = requests.get(data_url)
# Parse the HTML content using BeautifulSoup
html_content = BeautifulSoup(web_response.content, 'html.parser')
# Specify a key phrase from the chart caption to locate the right data
search_caption = "Proportion of people 15 years and over who were current daily smokers by age, 2011"  # update this as needed
# Initialize variable to store the desired div
target_div = None

# Loop through all div elements with the specified class
for container in html_content.find_all('div', {'class': 'chart-data-wrapper'}):
    # Extract the caption text
    chart_caption = container.find('pre', {'class': 'chart-caption'}).text
    # Check if the specified caption part is in the extracted caption
    if search_caption in chart_caption:
        target_div = container
        break
# Parse and extract chart data from JSON format
chart_data = json.loads(target_div.find('pre', {'class': 'chart-data'}).text)


desired_values = chart_data[7]
desired_values

[[1.6], [7.3], [10.9], [10.9], [13.8], [14.9], [8.7], [2.9]]

Creating ABS Smoking column

In [26]:
# Smoking rates by age group as extracted
smoking_rates = [item for sublist in desired_values for item in sublist]

# Define age bins corresponding to the age groups in the rate table
bins = [0, 17, 24, 34, 44, 54, 64, 74, 120]

# Assign each age to an age group
df['age_group_ABS'] = pd.cut(df['age'], bins=bins, labels=[0, 1, 2, 3, 4, 5, 6, 7], right=False)

# Function to impute NaN based on smoking probability
def impute_smoking(row):
    if pd.isna(row['smoke']):
        # Get the smoking rate for the age group
        rate = smoking_rates[int(row['age_group_ABS'])]
        # Generate a random number and compare to the smoking rate to decide on smoking status
        return 1 if np.random.rand() < rate / 100 else 0
    else:
        return row['smoke']

df['ABS smoke'] = df.apply(impute_smoking, axis=1)

df


,age,sex,painloc,painexer,cp,trestbps,smoke,fbs,prop,nitr,pro,diuretic,thaldur,thalach,exang,oldpeak,slope,target,age_group_ABS,ABS smoke
0,63.0,1.0,1.0,1.0,1.0,145.0,NaN,1.0,0.0,0.0,0.0,0.0,10.5,150.0,0.0,2.30000,3.0,0.0,5,0.0
1,67.0,1.0,1.0,1.0,4.0,160.0,NaN,0.0,1.0,0.0,0.0,0.0,9.5,108.0,1.0,1.50000,2.0,1.0,6,0.0
2,67.0,1.0,1.0,1.0,4.0,120.0,NaN,0.0,1.0,0.0,0.0,0.0,8.5,129.0,1.0,2.60000,2.0,1.0,6,0.0
3,37.0,1.0,1.0,1.0,3.0,130.0,NaN,0.0,1.0,0.0,0.0,0.0,13.0,187.0,0.0,3.50000,3.0,0.0,3,1.0
4,41.0,0.0,1.0,1.0,2.0,130.0,NaN,0.0,0.0,0.0,0.0,0.0,7.0,172.0,0.0,1.40000,1.0,0.0,3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,54.0,1.0,1.0,1.0,4.0,180.0,NaN,0.0,0.0,0.0,0.0,0.0,7.5,150.0,0.0,1.50000,2.0,1.0,5,1.0
895,56.0,1.0,1.0,1.0,4.0,125.0,1.0,1.0,0.0,0.0,0.0,0.0,6.5,103.0,1.0,1.00000,2.0,1.0,5,1.0
896,56.0,1.0,0.0,1.0,3.0,125.0,NaN,0.0,0.0,0.0,0.0,0.0,6.0,98.0,0.0,0.87049,2.0,1.0,5,1.0
897,54.0,1.0,1.0,1.0,4.0,130.0,NaN,0.0,0.0,0.0,0.0,0.0,9.5,110.0,1.0,3.00000,2.0,1.0,5,0.0


Code to Webscrape CDC URL

In [27]:
# Define the URL containing information on smoking statistics for various demographics.
source_url = "https://www.cdc.gov/tobacco/data_statistics/fact_sheets/adult_data/cig_smoking/index.htm"

# Fetch the data from the source.
server_response = requests.get(source_url)
if server_response.status_code != 200:
    print("Failed to retrieve data")

# Retrieve and parse the HTML content from the fetched data.
html_data = server_response.content
selector = Selector(text=html_data)
target_div = selector.xpath("//div[@class='row '][3]")

# Extracting lists from specific parts of the webpage.
list_selector = target_div.xpath("//ul[@class='block-list']")

# Extracting textual content from the lists related to gender smoking rates.
gender_data = list_selector[0].xpath(".//li/text()")
age_data = list_selector[1].xpath(".//li/text()")

# Parsing percentages for male and female from the extracted text.
male_rate = float(gender_data[0].extract().split("(")[1].split("%)")[0])
female_rate = float(gender_data[1].extract().split("(")[1].split("%)")[0])

# Creating a dictionary to store age-specific smoking rates.
age_rates = {}
for item in age_data:
    age_range = item.extract().split("aged ")[1].split(" years")[0]
    rate = float(item.extract().split("(")[1].split("%)")[0])
    if "–" in age_range:
        age_limits = age_range.split("–")
        age_rates[(int(age_limits[0]), int(age_limits[1]))] = rate
    else:
        age_rates[(int(age_range), float('inf'))] = rate

# Adjust male smoking rates based on comparison with female rates.
adjusted_male_rates = {key: value * (male_rate / female_rate) for key, value in age_rates.items()}

print("Adjusted male smoking rates by age:", adjusted_male_rates)
print("Female smoking rates by age:", age_rates)


Adjusted male smoking rates by age: {(18, 24): 6.874257425742575, (25, 44): 16.342574257425742, (45, 64): 19.325742574257426, (65, inf): 10.765346534653467}
Female smoking rates by age: {(18, 24): 5.3, (25, 44): 12.6, (45, 64): 14.9, (65, inf): 8.3}


Creating CDC Smoking column

In [28]:
bins = [18, 24, 44, 64, float('inf')]
labels = [(18, 24), (25, 44), (45, 64), (65, float('inf'))]

df['age_group_CDC'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)

def impute_smoking(row):
    if pd.isna(row['smoke']):
        age_group = row['age_group_CDC']
        if row['sex'] == 1:  # Male
            return 1 if np.random.rand() < adjusted_male_rates[age_group] / 100 else 0
        else:  # Female
            return 1 if np.random.rand() < age_rates[age_group] / 100 else 0
    else:
        return row['smoke']

df['CDC smoke'] = df.apply(impute_smoking, axis=1)

df




,age,sex,painloc,painexer,cp,trestbps,smoke,fbs,prop,nitr,...,thaldur,thalach,exang,oldpeak,slope,target,age_group_ABS,ABS smoke,age_group_CDC,CDC smoke
0,63.0,1.0,1.0,1.0,1.0,145.0,NaN,1.0,0.0,0.0,...,10.5,150.0,0.0,2.30000,3.0,0.0,5,0.0,"(45, 64)",0.0
1,67.0,1.0,1.0,1.0,4.0,160.0,NaN,0.0,1.0,0.0,...,9.5,108.0,1.0,1.50000,2.0,1.0,6,0.0,"(65, inf)",0.0
2,67.0,1.0,1.0,1.0,4.0,120.0,NaN,0.0,1.0,0.0,...,8.5,129.0,1.0,2.60000,2.0,1.0,6,0.0,"(65, inf)",1.0
3,37.0,1.0,1.0,1.0,3.0,130.0,NaN,0.0,1.0,0.0,...,13.0,187.0,0.0,3.50000,3.0,0.0,3,1.0,"(25, 44)",1.0
4,41.0,0.0,1.0,1.0,2.0,130.0,NaN,0.0,0.0,0.0,...,7.0,172.0,0.0,1.40000,1.0,0.0,3,0.0,"(25, 44)",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,54.0,1.0,1.0,1.0,4.0,180.0,NaN,0.0,0.0,0.0,...,7.5,150.0,0.0,1.50000,2.0,1.0,5,1.0,"(45, 64)",0.0
895,56.0,1.0,1.0,1.0,4.0,125.0,1.0,1.0,0.0,0.0,...,6.5,103.0,1.0,1.00000,2.0,1.0,5,1.0,"(45, 64)",1.0
896,56.0,1.0,0.0,1.0,3.0,125.0,NaN,0.0,0.0,0.0,...,6.0,98.0,0.0,0.87049,2.0,1.0,5,1.0,"(45, 64)",0.0
897,54.0,1.0,1.0,1.0,4.0,130.0,NaN,0.0,0.0,0.0,...,9.5,110.0,1.0,3.00000,2.0,1.0,5,0.0,"(45, 64)",0.0


Imputing original 'smoke' column using ABS and CDC smoke columns.

Method:
for NaN values, if both ABS smoke and CDC smoke are 0, set row to 0

If at least one of ABS smoke or CDC smoke is 1, set row to 1.

In [29]:
# Define the function to impute the 'smoke' column
def impute_smoke(row):
    if pd.isna(row['smoke']):
        if row['ABS smoke'] == 0 and row['CDC smoke'] == 0:
            return 0
        else:
            return 1
    else:
        return row['smoke']

# Apply the function across each row
df['smoke'] = df.apply(impute_smoke, axis=1)
df

,age,sex,painloc,painexer,cp,trestbps,smoke,fbs,prop,nitr,...,thaldur,thalach,exang,oldpeak,slope,target,age_group_ABS,ABS smoke,age_group_CDC,CDC smoke
0,63.0,1.0,1.0,1.0,1.0,145.0,0.0,1.0,0.0,0.0,...,10.5,150.0,0.0,2.30000,3.0,0.0,5,0.0,"(45, 64)",0.0
1,67.0,1.0,1.0,1.0,4.0,160.0,0.0,0.0,1.0,0.0,...,9.5,108.0,1.0,1.50000,2.0,1.0,6,0.0,"(65, inf)",0.0
2,67.0,1.0,1.0,1.0,4.0,120.0,1.0,0.0,1.0,0.0,...,8.5,129.0,1.0,2.60000,2.0,1.0,6,0.0,"(65, inf)",1.0
3,37.0,1.0,1.0,1.0,3.0,130.0,1.0,0.0,1.0,0.0,...,13.0,187.0,0.0,3.50000,3.0,0.0,3,1.0,"(25, 44)",1.0
4,41.0,0.0,1.0,1.0,2.0,130.0,1.0,0.0,0.0,0.0,...,7.0,172.0,0.0,1.40000,1.0,0.0,3,0.0,"(25, 44)",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,54.0,1.0,1.0,1.0,4.0,180.0,1.0,0.0,0.0,0.0,...,7.5,150.0,0.0,1.50000,2.0,1.0,5,1.0,"(45, 64)",0.0
895,56.0,1.0,1.0,1.0,4.0,125.0,1.0,1.0,0.0,0.0,...,6.5,103.0,1.0,1.00000,2.0,1.0,5,1.0,"(45, 64)",1.0
896,56.0,1.0,0.0,1.0,3.0,125.0,1.0,0.0,0.0,0.0,...,6.0,98.0,0.0,0.87049,2.0,1.0,5,1.0,"(45, 64)",0.0
897,54.0,1.0,1.0,1.0,4.0,130.0,0.0,0.0,0.0,0.0,...,9.5,110.0,1.0,3.00000,2.0,1.0,5,0.0,"(45, 64)",0.0


Step 1: Data Preparation and Splitting

In [30]:
df = df.drop('age_group_ABS', axis=1)
df = df.drop('age_group_CDC', axis=1)

from sklearn.model_selection import train_test_split

# Assume df is your DataFrame and 'target' is the column with 0 or 1
X = df.drop('target', axis=1)  # Features
y = df['target']  # Target
print(X)

# Splitting the data with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)
nan_counts = X.isna().sum()

# This will give you a Series with the count of NaNs in each column
print(nan_counts)


      age  sex  painloc  painexer   cp  trestbps  smoke  fbs  prop  nitr  pro  \
0    63.0  1.0      1.0       1.0  1.0     145.0    0.0  1.0   0.0   0.0  0.0   
1    67.0  1.0      1.0       1.0  4.0     160.0    0.0  0.0   1.0   0.0  0.0   
2    67.0  1.0      1.0       1.0  4.0     120.0    1.0  0.0   1.0   0.0  0.0   
3    37.0  1.0      1.0       1.0  3.0     130.0    1.0  0.0   1.0   0.0  0.0   
4    41.0  0.0      1.0       1.0  2.0     130.0    1.0  0.0   0.0   0.0  0.0   
..    ...  ...      ...       ...  ...       ...    ...  ...   ...   ...  ...   
894  54.0  1.0      1.0       1.0  4.0     180.0    1.0  0.0   0.0   0.0  0.0   
895  56.0  1.0      1.0       1.0  4.0     125.0    1.0  1.0   0.0   0.0  0.0   
896  56.0  1.0      0.0       1.0  3.0     125.0    1.0  0.0   0.0   0.0  0.0   
897  54.0  1.0      1.0       1.0  4.0     130.0    0.0  0.0   0.0   0.0  0.0   
898  66.0  0.0      1.0       1.0  4.0     155.0    0.0  0.0   0.0   0.0  0.0   

     diuretic  thaldur  tha

Step 2: Model Training and Cross-Validation


In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Setting up the logistic regression with hyperparameter grid
log_reg_params = {'C': [0.01, 0.1, 1, 10, 100]}
log_reg = LogisticRegression(solver='liblinear')

log_reg_grid = GridSearchCV(log_reg, log_reg_params, cv=5, scoring='accuracy')
log_reg_grid.fit(X_train, y_train)
print("Best parameters for Logistic Regression:", log_reg_grid.best_params_)
print("Cross-validated accuracy:", log_reg_grid.best_score_)

# Setting up the random forest classifier with hyperparameter grid
rf_params = {'n_estimators': [10, 50, 100, 200], 'max_depth': [None, 10, 20, 30]}
rf = RandomForestClassifier()

rf_grid = GridSearchCV(rf, rf_params, cv=5, scoring='accuracy')
rf_grid.fit(X_train, y_train)
print("Best parameters for Random Forest:", rf_grid.best_params_)
print("Cross-validated accuracy:", rf_grid.best_score_)


Best parameters for Logistic Regression: {'C': 0.1}
Cross-validated accuracy: 0.8084272678475577
Best parameters for Random Forest: {'max_depth': None, 'n_estimators': 100}
Cross-validated accuracy: 0.8121463077984817


Step 3: Model Selection


In [32]:
# Compare the performance and select the best model
if log_reg_grid.best_score_ > rf_grid.best_score_:
    final_model = log_reg_grid.best_estimator_
    print("Selected Logistic Regression as the final model.")
else:
    final_model = rf_grid.best_estimator_
    print("Selected Random Forest as the final model.")

# Final evaluation on the test data
y_pred = final_model.predict(X_test)
print("Performance on the test set:")
print(classification_report(y_test, y_pred))


Selected Random Forest as the final model.
Performance on the test set:
              precision    recall  f1-score   support

         0.0       0.73      0.68      0.70        40
         1.0       0.75      0.80      0.78        50

    accuracy                           0.74        90
   macro avg       0.74      0.74      0.74        90
weighted avg       0.74      0.74      0.74        90

